In [7]:
!pip install feast[aws]==0.19.3

In [ ]:
!git clone <feature_repo>

## **Apply Feature definition**
Before running the next set of commands do the following.
*   Create entity and feature defintions
*   Creating all the resources and mapping external database


In [3]:
### Contents of rfm_features.py 

### Uncomment the following code block copy and create file called rfm_features.py in the root folder of feature_repo. 


# from feast import Entity, ValueType, FeatureView, Feature, RedshiftSource
# from datetime import timedelta

# # Customer ID entity definition.
# customer = Entity(
#     name='customer',
#     value_type=ValueType.STRING,
#     join_key='customerid',
#     description="Id of the customer"
# )

# # Redshift batch source
# rfm_features_source = RedshiftSource(
#     query="SELECT * FROM spectrum.customer_rfm_features",
#     event_timestamp_column="event_timestamp",
#     created_timestamp_column="created_timestamp",
# )

# rfm_features_features = FeatureView(
#     name="customer_rfm_features",

#     entities=["customer"],
#     ttl=timedelta(days=3650),
#     features=[
#         Feature(name="recency", dtype=ValueType.INT32),
#         Feature(name="frequency", dtype=ValueType.INT32),
#         Feature(name="monetaryvalue", dtype=ValueType.DOUBLE),
#         Feature(name="r", dtype=ValueType.INT32),
#         Feature(name="f", dtype=ValueType.INT32),
#         Feature(name="m", dtype=ValueType.INT32),
#         Feature(name="rfmscore", dtype=ValueType.INT32),
#         Feature(name="revenue6m", dtype=ValueType.DOUBLE),
#         Feature(name="ltvcluster", dtype=ValueType.INT32),
#         Feature(name="segmenthighvalue", dtype=ValueType.INT32),
#         Feature(name="segmentlowvalue", dtype=ValueType.INT32),
#         Feature(name="segmentmidvalue", dtype=ValueType.INT32),
#     ],
#     batch_source=rfm_features_source,
# )


In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "<aws_key>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws_secret>"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [ ]:
%cd customer_segmentation/
!feast apply

[Errno 2] No such file or directory: 'customer_segmentation/'
/content/customer_segmentation
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,
03/14/2022 03:01:38 AM INFO:Found credentials in environment variables.
Created data source 
Created entity customer
Created feature view customer_rfm_features

Deploying infrastructure for customer_rfm_features


## **Check-in Feature repo**
After running `apply` command check in the feature repo back to GitHub so that it is available for later.

## **After Feast Apply**

In [1]:
%cd customer_segmentation/
from feast import FeatureStore
store = FeatureStore(repo_path=".")

/content/customer_segmentation
Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]


In [2]:
print("-----------------------Entity----------------------------")
for entity in store.list_entities():
  print(f"entity: {entity}")
print("--------------------Feature Views----------------------------")
for feature_view in store.list_feature_views():
  print(f"List of FeatureViews: {feature_view}")

-----------------------Entity----------------------------
entity: {
  "spec": {
    "name": "customer",
    "valueType": "STRING",
    "description": "Id of the customer",
    "joinKey": "customerid"
  },
  "meta": {
    "createdTimestamp": "2022-03-27T18:07:14.597257Z",
    "lastUpdatedTimestamp": "2022-03-27T18:07:14.597257Z"
  }
}
--------------------Feature Views----------------------------
List of FeatureViews: {
  "spec": {
    "name": "customer_rfm_features",
    "entities": [
      "customer"
    ],
    "features": [
      {
        "name": "recency",
        "valueType": "INT32"
      },
      {
        "name": "frequency",
        "valueType": "INT32"
      },
      {
        "name": "monetaryvalue",
        "valueType": "DOUBLE"
      },
      {
        "name": "r",
        "valueType": "INT32"
      },
      {
        "name": "f",
        "valueType": "INT32"
      },
      {
        "name": "m",
        "valueType": "INT32"
      },
      {
        "name": "rfmscore",
    

/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,


## **Query data after ingestion**

In [3]:
import pandas as pd
from datetime import datetime, timedelta
entity_df = pd.DataFrame.from_dict(
    {
        "customerid": ["12747.0", "12748.0", "12749.0"],
        "event_timestamp": [datetime.now()]*3
    }
)
entity_df.head()

In [ ]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
              "customer_rfm_features:recency", 
              "customer_rfm_features:frequency", 
              "customer_rfm_features:monetaryvalue", 
              "customer_rfm_features:r", 
              "customer_rfm_features:f", 
              "customer_rfm_features:m"]
    )

/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,


In [ ]:
df = job.to_df()

In [ ]:
df.head()

,customerid,event_timestamp,recency,frequency,monetaryvalue,r,f,m
0,12747.0,2022-03-15 00:58:45.594038,7,35,1082.09,3,2,3
1,12749.0,2022-03-15 00:58:45.594038,8,54,782.10,3,3,3
2,12748.0,2022-03-15 00:58:45.594038,1,582,4336.73,3,3,3
